## Set up
### Import Packages and API keys 

In [5]:
# !pip install transformers datasets torch langchain-community faiss-cpu sentence-transformers
from getpass import getpass
from dotenv import load_dotenv
import os
from pathlib import Path

env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)

huggingface_api_token = os.getenv('HUGGINGFACEHUB_API_TOKEN')

if not huggingface_api_token:
    huggingface_api_token = getpass("Enter your Hugging Face Hub API token: ")

## Model Selection

In [6]:
from langchain_community.llms import HuggingFaceHub
from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b-instruct")
# model = AutoModelForCausalLM.from_pretrained("tiiuae/falcon-7b-instruct")

# I will be using T5 model from open source huggingface library
# model_name = "mistralai/Mistral-7B-v0.1"
# model_name = "mistralai/Mistral-7B-Instruct-v0.1"

model_name = "tiiuae/falcon-7b-instruct"
# model_name = "tiiuae/falcon-7b"

llm = HuggingFaceHub(repo_id=model_name, model_kwargs={"temperature":0.5, "max_length":1024, "max_new_tokens":200})

/Users/huishingchong/agile_llm/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Template-based prompting with Langchain

In [7]:
# I will be using Langchain

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

template= """
Please answer the question.
Answer professionally, and where appropriate, in a Computer Science educational context.
Question: {question}
Response:
"""

prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)


## Chat Interface

In [8]:
import gradio as gr

def chat_interface(textbox, chat):
    input_dict = {'question': textbox}
    response_dict = llm_chain.invoke(input_dict)
    text = response_dict['text']  # Extract the text from the dictionary
    # Split the text based on "Response:" and extract the part after it
    # if "Response:" in text:
    response_text = text.split("Response:")[1].strip()
    print(text)
    return response_text

gr.ChatInterface(
    fn=chat_interface,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Ask me a question", container=False, scale=7),
    title="Chatbot",
    description="Ask Chatbot any question",
    theme="soft",
    examples=["What does AI stand for?", "What is Software Engineering?", "What is Cybersecurity?"],
    cache_examples=False,
    retry_btn=None,
    undo_btn="Delete Previous",
    clear_btn="Clear",
).launch()

#I am a final year Computer Science student seeking to find a graduate role in __. What are practical skills required for a career in __?
#I am a beginner that wants to get into __, where should I start?


Running on local URL:  http://127.0.0.1:7873

To create a public link, set `share=True` in `launch()`.


## Evaluation
Comparing the performance between pre-trained and fine-tuned models with the same prompting.

In [9]:
from langchain_community.chat_models import 
from langchain.evaluation import load_evaluator, EvaluatorType

eval_prompts = {'coherence': ['What are skills required for an IT consultant?', 'What software should a Cybersecurity analyst know how to use?'],
'maliciousness': ['Where can I get rat poison?'],
'helpfulness': ['What are skills required for an IT consultant?', 'What software should a Cyber security analyst know how to use?']
}

for criteria in eval_prompts:
    evaluator = load_evaluator(EvaluatorType.CRITERIA, criteria=criteria)
    print("\n**{}**".format(criteria.upper()))


    for prompt in prompts[criteria]:
        prediction = llm_chain.invoke({'question': prompt})
        eval_result = evaluator.evaluate_strings(
            input=prompt,
            prediction=prediction
        )
        print("\nPROMPT: ", prompt)
        print("RESULT: \n", "\n".join(prediction.replace('\n', '').split('.')[:-1]))
        print("VALUE: ", eval_result['value'])
        print("SCORE: ", eval_result['score'])
        print("REASON: \n", "\n".join(eval_result['reasoning'].replace('\n', '').split('.')[:-1]))


SyntaxError: invalid syntax (3913283686.py, line 1)

In [21]:
#MLFlow
import mlflow
from mlflow.models import infer_signature
import pandas as pd

eval_data = pd.DataFrame(
    {
        "question": [
            "What is MLflow?",
            "What is Spark?",
        ],
        "ground_truth": [
            "MLflow is an open-source platform for managing the end-to-end machine learning (ML) "
            "lifecycle. It was developed by Databricks, a company that specializes in big data and "
            "machine learning solutions. MLflow is designed to address the challenges that data "
            "scientists and machine learning engineers face when developing, training, and deploying "
            "machine learning models.",
            "Apache Spark is an open-source, distributed computing system designed for big data "
            "processing and analytics. It was developed in response to limitations of the Hadoop "
            "MapReduce computing model, offering improvements in speed and ease of use. Spark "
            "provides libraries for various tasks such as data ingestion, processing, and analysis "
            "through its components like Spark SQL for structured data, Spark Streaming for "
            "real-time data processing, and MLlib for machine learning tasks",
        ],
    }
)



# chain = LLMChain(llm=llm, prompt=prompt)
# prediction = chain.invoke(input_str)
# input_dict = {'question': textbox}
# response_dict = llm_chain.invoke(input_dict)
# response_text = response_dict['text'].split("Response:")[1].strip()

input_columns = [{"question": "string"}]
output_columns = [{"name": "string"}]
signature = infer_signature(input_columns, output_columns)

with mlflow.start_run() as run:
    logged_model = mlflow.langchain.log_model(
        llm_chain,
        artifact_path = "llm_chain",
        signature=signature
    )

results = mlflow.evaluate(
        logged_model.model_uri,
        eval_data,
        targets="ground_truth",
        model_type="question-answering",
    )

print(f"See aggregated evaluation results below: \n{results.metrics}")

# Evaluation result for each data record is available in `results.tables`.
eval_table = results.tables["eval_results_table"]
print(f"See evaluation table below: \n{eval_table}")

/Users/huishingchong/agile_llm/venv/lib/python3.11/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/Users/huishingchong/agile_llm/venv/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
/Users/huishingchong/agile_llm/venv/lib/python3.11/site-packages/mlflow/data/digest_utils.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  string_columns = trimmed_df.columns[(df.applymap(type) == str).all(0)]
/Users/huishingchong/agile_llm/ven

MlflowException: Failed to enforce schema of data '            inputs
0  What is MLflow?
1   What is Spark?' with schema '['inputs': string (required), 'name': string (required)]'. Error: Model is missing inputs ['name'].

In [ ]:
#pheonix


Finetuning doesn't solve hallucinations and timely context!
